In [1]:
!pip install signate

from googleapiclient.discovery import build
import io, os
from googleapiclient.http import MediaIoBaseDownload
from google.colab import auth


auth.authenticate_user()

drive_service = build('drive', 'v3')
results = drive_service.files().list(
        q="name = 'signate.json'", fields="files(id)").execute()
signate_api_key = results.get('files', [])

filename = "/root/.signate/signate.json"
os.makedirs(os.path.dirname(filename), exist_ok=True)

request = drive_service.files().get_media(fileId=signate_api_key[0]['id'])
fh = io.FileIO(filename, 'wb')
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))
os.chmod(filename, 600)

     |████████████████████████████████| 138 kB 8.7 MB/s 
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9672 sha256=69a0cf3a9ba7ce1283d1536e34f884ac130123adce2e009a5e04675f7eea3671
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.7 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


Download 100%.


In [2]:
! signate list
! signate files --competition-id=492
! signate download --competition-id=492

  competitionId  title                                                                  closing     prize        submitters
---------------  ---------------------------------------------------------------------  ----------  ---------  ------------
              1  【練習問題】銀行の顧客ターゲティング                                   -                              5179
             24  【練習問題】お弁当の需要予測                                           -                              6355
             27  【練習問題】Jリーグの観客動員数予測                                    -                              1433
            100  【練習問題】手書き文字認識                                             -           Knowledge           175
            102  【練習問題】タイタニックの生存予測                                     -           Knowledge          1065
            103  【練習問題】音楽ラベリング                                             -           Knowledge            61
            104  【練習問題】スパムメール分類                                           -           Knowledge     

In [3]:
# ディレクトリ内のデータの確認
%ls

adc.json                                       requirements.txt
category_names.csv                             sales_history.csv
item_categories.csv                            sample_data/
PBL01_h_exercise4_question.pdf                 sample_submission.csv
PBL01_i_exercise4_supplementary-documents.pdf  test.csv
PBL01_sample_code.ipynb


In [4]:
# Googleドライブのマウント
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [8]:
# ライブラリのインポート
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler

# データの読み込み
pre_train = pd.read_csv('/content/drive/My Drive/AIQuest2021/PBL_01_需要予測/Phase2/demand/processed_data/pre_train.csv')
test = pd.read_csv('test.csv')
category_names = pd.read_csv('category_names.csv')
item_categories = pd.read_csv('item_categories.csv')
sales_history = pd.read_csv('sales_history.csv')
time_group = pd.read_csv('/content/drive/My Drive/AIQuest2021/PBL_01_需要予測/Phase2/demand/processed_data/time_group.csv')

'''
テストデータの加工
'''

# カラム名の変更
test = test.rename(columns={'商品ID': 'id',
                            '店舗ID':'store_id'
                            })

# month_biningカラムの作成
test['month_bining'] = 23

'''
学習データとテストデータの結合
'''

df = pd.concat([pre_train, test], sort=False).reset_index(drop=True)

# 不要なカラムの削除
df = df.drop(['index'], axis=1)

'''
返品データの修正
'''

# 返品データを修正する
for i in range(len(df)):
    if df['sales'][i] < 0: # train['sales']のi行目の要素抽出
        df['sales'][i] = 0

print(df['sales'].value_counts())
print(df['sales'].isnull().sum())

'''
ラグ特徴量の作成
'''

# 12ヶ月前の売上データ
lag12before = df.copy()
lag12before['month_bining'] = lag12before['month_bining']+12
lag12before = lag12before.rename(columns={'sales':'sales_before_12'})

# 11ヶ月前の売上データ
lag11before = df.copy()
lag11before['month_bining'] = lag11before['month_bining']+11
lag11before = lag11before.rename(columns={'sales':'sales_before_11'})

# 10ヶ月前の売上データ
lag10before = df.copy()
lag10before['month_bining'] = lag10before['month_bining']+10
lag10before = lag10before.rename(columns={'sales':'sales_before_10'})

# 9ヶ月前の売上データ
lag9before = df.copy()
lag9before['month_bining'] = lag9before['month_bining']+9
lag9before = lag9before.rename(columns={'sales':'sales_before_9'})

# 8ヶ月前の売上データ
lag8before = df.copy()
lag8before['month_bining'] = lag8before['month_bining']+8
lag8before = lag8before.rename(columns={'sales':'sales_before_8'})

# 7ヶ月前の売上データ
lag7before = df.copy()
lag7before['month_bining'] = lag7before['month_bining']+7
lag7before = lag7before.rename(columns={'sales':'sales_before_7'})

# 6ヶ月前の売上データ
lag6before = df.copy()
lag6before['month_bining'] = lag6before['month_bining']+6
lag6before = lag6before.rename(columns={'sales':'sales_before_6'})

# 5ヶ月前の売上データ
lag5before = df.copy()
lag5before['month_bining'] = lag5before['month_bining']+5
lag5before = lag5before.rename(columns={'sales':'sales_before_5'})

# 4ヶ月前の売上データ
lag4before = df.copy()
lag4before['month_bining'] = lag4before['month_bining']+4
lag4before = lag4before.rename(columns={'sales':'sales_before_4'})

# 3ヶ月前の売上データ
lag3before = df.copy()
lag3before['month_bining'] = lag3before['month_bining']+3
lag3before = lag3before.rename(columns={'sales':'sales_before_3'})

# 2ヶ月前の売上データ
lag2before = df.copy()
lag2before['month_bining'] = lag2before['month_bining']+2
lag2before = lag2before.rename(columns={'sales':'sales_before_2'})

# ラグ特徴量の追加
df = pd.merge(df, lag12before, on=['month_bining', 'id', 'store_id'], how='left')
df = pd.merge(df, lag11before, on=['month_bining', 'id', 'store_id'], how='left')
df = pd.merge(df, lag10before, on=['month_bining', 'id', 'store_id'], how='left')
df = pd.merge(df, lag9before, on=['month_bining', 'id', 'store_id'], how='left')
df = pd.merge(df, lag8before, on=['month_bining', 'id', 'store_id'], how='left')
df = pd.merge(df, lag7before, on=['month_bining', 'id', 'store_id'], how='left')
df = pd.merge(df, lag6before, on=['month_bining', 'id', 'store_id'], how='left')
df = pd.merge(df, lag5before, on=['month_bining', 'id', 'store_id'], how='left')
df = pd.merge(df, lag4before, on=['month_bining', 'id', 'store_id'], how='left')
df = pd.merge(df, lag3before, on=['month_bining', 'id', 'store_id'], how='left')
df = pd.merge(df, lag2before, on=['month_bining', 'id', 'store_id'], how='left')
print(df.groupby('month_bining').agg({'sales_before_12': 'count'}))

'''
商品カテゴリ、年月の情報を付与
'''

# カラム名の変更
category_names = category_names.rename(columns={'商品カテゴリID':'category_id',
                                                '商品カテゴリ名':'category_name'})

item_categories = item_categories.rename(columns={'商品ID': 'id',
                                                  '商品カテゴリID':'category_id'})

sales_history = sales_history.rename(columns={'商品ID': 'id',
                                              '日付':'datetime',
                                              '店舗ID':'store_id',
                                              '商品価格':'price',
                                              '売上個数':'sales'
                                              })

# 商品カテゴリIDの付与
df = pd.merge(df, item_categories, on='id', how='left')

# 商品カテゴリ名の付与
df = pd.merge(df, category_names, on='category_id', how='left')

# カテゴリを分ける
df['category'] = df['category_name'].apply(lambda x : x.split(' - ')[0])
df['category_detail'] = df['category_name'].apply(lambda x : x.split(' - ')[1])

# 年、月の付与
df = pd.merge(df, time_group, on='month_bining', how='left')

'''
売値の情報を付与
（最終的に消す）
'''

# 商品IDごとの売値（平均値）を集計する
selling_price = sales_history.groupby('id').agg({'price':np.mean}).reset_index()

# 売値の付与
df = pd.merge(df, selling_price, on='id', how='left')

'''
売上数の平均の情報を付与
'''

# 商品IDごとの平均売上数
sales_mean = sales_history.groupby('id').agg({'sales':np.mean}).reset_index()

# カラム名の変更
sales_mean = sales_mean.rename(columns={'sales':'sales_mean'})

# 平均売上数を付与
df = pd.merge(df, sales_mean, on='id', how='left')

'''
売上高データの作成
'''

# 売上データの作成
sales_history['earnings'] = sales_history['price'] * sales_history['sales']

'''
店舗ごとの総売上高の情報を付与
'''

# 店舗の総売上
store_earnings = sales_history.groupby('store_id').agg({'earnings':np.sum}).reset_index()

# カラム名の変更
store_earnings = store_earnings.rename(columns={'earnings':'store_earnings'})

# 店舗の総売上を付与
df = pd.merge(df, store_earnings, on='store_id', how='left')

'''
欠損値の補完
'''

# 欠損値の確認
print(df.isnull().sum())

df['sales_before_12'] = df['sales_before_12'].fillna(0)
df['sales_before_11'] = df['sales_before_11'].fillna(0)
df['sales_before_10'] = df['sales_before_10'].fillna(0)
df['sales_before_9'] = df['sales_before_9'].fillna(0)
df['sales_before_8'] = df['sales_before_8'].fillna(0)
df['sales_before_7'] = df['sales_before_7'].fillna(0)
df['sales_before_6'] = df['sales_before_6'].fillna(0)
df['sales_before_5'] = df['sales_before_5'].fillna(0)
df['sales_before_4'] = df['sales_before_4'].fillna(0)
df['sales_before_3'] = df['sales_before_3'].fillna(0)
df['sales_before_2'] = df['sales_before_2'].fillna(0)
print(df.isnull().sum())

'''
不要なカラムの削除
'''

# category_nameカラムの削除
df = df.drop('category_name', axis=1)

'''
データフレームの保存
'''

processed_train_test_df = df.to_csv('/content/drive/My Drive/AIQuest2021/PBL_01_需要予測/Phase2/demand/processed_data/processed_train_test_df.csv', header=True, index=False)

0.0      1154748
1.0       288980
2.0        91331
3.0        38995
4.0        20583
          ...   
159.0          1
166.0          1
170.0          1
122.0          1
417.0          1
Name: sales, Length: 175, dtype: int64
3060
              sales_before_12
month_bining                 
0.0                         0
1.0                         0
2.0                         0
3.0                         0
4.0                         0
5.0                         0
6.0                         0
7.0                         0
8.0                         0
9.0                         0
10.0                        0
11.0                        0
12.0                    42354
13.0                    40356
14.0                    40122
15.0                    37296
16.0                    36216
17.0                    36774
18.0                    37746
19.0                    35406
20.0                    31392
21.0                    32508
23.0                     2700
month_bining       

In [7]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1651986 entries, 0 to 1651985
Data columns (total 23 columns):
 #   Column           Non-Null Count    Dtype  
---  ------           --------------    -----  
 0   month_bining     1651986 non-null  float64
 1   id               1651986 non-null  float64
 2   store_id         1651986 non-null  float64
 3   sales            1648926 non-null  float64
 4   sales_before_12  1651986 non-null  float64
 5   sales_before_11  1651986 non-null  float64
 6   sales_before_10  1651986 non-null  float64
 7   sales_before_9   1651986 non-null  float64
 8   sales_before_8   1651986 non-null  float64
 9   sales_before_7   1651986 non-null  float64
 10  sales_before_6   1651986 non-null  float64
 11  sales_before_5   1651986 non-null  float64
 12  sales_before_4   1651986 non-null  float64
 13  sales_before_3   1651986 non-null  float64
 14  sales_before_2   1651986 non-null  float64
 15  category_id      1651986 non-null  int64  
 16  category         1